In [ ]:
import pandas as pd
import cv2
import opencv_jupyter_ui as jcv2
from train_model import train_model
from feat import Detector
import warnings
import time
warnings.filterwarnings(action="ignore", category=FutureWarning)
from furhat_remote_api import FurhatRemoteAPI
from time import sleep
import speech_recognition as sr

In [ ]:
model = train_model()
detector = Detector(device="cuda")

In [16]:
cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_BUFFERSIZE, 1)

last_detection_time = time.time()
emotion = None

emotion_detection_active = False

# Function to start emotion detection
def start_emotion_detection():
    global emotion_detection_active
    emotion_detection_active = True

# Function to stop emotion detection
def stop_emotion_detection():
    global emotion_detection_active
    emotion_detection_active = False

#Furhat code

# Furhat IP address
FURHAT_IP = "127.0.1.1"

# Connect to Furhat
furhat = FurhatRemoteAPI(FURHAT_IP)
furhat.set_led(red=100, green=50, blue=50)

# Furhat faces and voices
FACES = {'TheJoker': 'James'}
VOICES_EN = {'TheJoker': 'Matthew'}

# Furhat speech
def bsay(line):
    furhat.say(text=line, blocking=True)
    sleep(1)  # Add a delay of 1 second after each Furhat command

# Speech recognition setup
recognizer = sr.Recognizer()
with sr.Microphone() as source:
    recognizer.adjust_for_ambient_noise(source)

# Function to recognize speech
def recognize_speech():
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source, timeout=5)
    try:
        text = recognizer.recognize_google(audio)
        print("You said:", text)
        return text
    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio.")
        return None
    
# Function to react to speech
def react_to_speech(text):
    if text:
        print(f"You: {text}")
        if "hello" in text.lower():
            bsay("Hello, how are you?")
            furhat.gesture(name='Surprise')

            

# Function to react to emotion
def react_to_emotion(detected_emotion):
            
            
            print(f"Detected emotion: {detected_emotion}")
    # Adjust Furhat's behavior based on the detected emotion
            if detected_emotion == "happy":
                bsay("you are happy like Felix")
                furhat.gesture(name='HappyGesture')
            elif detected_emotion == "sad":
                bsay("you are sad like isak")
                furhat.gesture(name='SadGesture')
            elif detected_emotion == "neutral":
                bsay("You are neutral")
                furhat.gesture(name='Wink')     
            elif detected_emotion == "surprise":
                bsay("You are surprised")
                furhat.gesture(name='Wink') 
        

furhat.set_face(character=FACES['TheJoker'], mask="Adult")
furhat.set_voice(name=VOICES_EN['TheJoker'])
bsay("Hi")
furhat.gesture(name='ExpressDisgust')
while True:
    ret, frame = cam.read()
    if not ret:
        print("OpenCV found an error reading the next frame.")
        break

    current_time = time.time()  # Use a different variable name
    
    if current_time - last_detection_time >= 5:
        faces = detector.detect_faces(frame)
        landmarks = detector.detect_landmarks(frame, faces)    
        action_units = detector.detect_aus(frame, landmarks)
        action_units = action_units[0]
    
        # Predict emotion
        if emotion_detection_active:
            emotion = model.predict(action_units)
            react_to_emotion(emotion)

        talan=react_to_speech(recognize_speech())
        if talan:
            print(f"Recognized: {talan}")
            if "start" in talan.lower():
                start_emotion_detection()
                bsay("Emotion detection started")
            elif "stop" in talan.lower():
                stop_emotion_detection()
                bsay("Emotion detection stopped")    

        last_detection_time = current_time
    if emotion is not None:
        cv2.putText(frame, emotion[0], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    jcv2.imshow("Emotion Detection", frame)

    key = jcv2.waitKey(1) & 0xFF
    if key == 27: # ESC pressed
        break

cam.release()
jcv2.destroyAllWindows()

c:\Users\isakl\anaconda3\envs\IIS\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Listening...
You said: start
You: start


c:\Users\isakl\anaconda3\envs\IIS\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Listening...
Speech Recognition could not understand audio.


c:\Users\isakl\anaconda3\envs\IIS\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


...

c:\Users\isakl\anaconda3\envs\IIS\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Listening...


c:\Users\isakl\anaconda3\envs\IIS\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Listening...


KeyboardInterrupt: 